In [1]:
import numpy as np
import pickle
import os
from tqdm.notebook import tqdm

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from nlpsig_networks.scripts.swnu_network_functions import (
    swnu_network_hyperparameter_search,
)

In [4]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [5]:
%run ../load_anno_mi.py

In [6]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime,speaker
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-11-09 00:00:13,-1
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-11-09 00:00:24,1
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-11-09 00:00:25,-1
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-11-09 00:00:34,1
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-11-09 00:00:34,-1


In [7]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)

sbert_embeddings.shape

(9699, 384)

# SWNU Network

## Obtaining path by looking at post history

We can obtain a path by looking at the history of each post. Here we look at the last 10 posts (and pad with vectors of zeros if there are less than 10 posts) including the current post.

We only want to consider paths that correspond to a client's utterance as we want to model a change in mood at that time. Their history will still contain the therapist's utterances too.

In [8]:
features = ["time_encoding", "timeline_index"]
standardise_method = ["z_score", None]
include_features_in_path = True
include_features_in_input = True

In [9]:
num_epochs = 100
dimensions = [15]
swnu_hidden_dim_sizes_and_sig_depths = [([12], 3), ([10], 3)]
ffn_hidden_dim_sizes = [[32,32], [128,128], [512,512]]
dropout_rates = [0.1]
learning_rates = [5e-4, 3e-4, 1e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 3

In [10]:
kwargs = {
    "num_epochs": num_epochs,
    "df": anno_mi,
    "id_column": "transcript_id",
    "label_column": "client_talk_type",
    "embeddings": sbert_embeddings,
    "y_data": y_data_client,
    "output_dim": output_dim_client,
    "dimensions": dimensions,
    "log_signature": True,
    "pooling": "signature",
    "swnu_hidden_dim_sizes_and_sig_depths": swnu_hidden_dim_sizes_and_sig_depths,
    "ffn_hidden_dim_sizes": ffn_hidden_dim_sizes,
    "dropout_rates": dropout_rates,
    "learning_rates": learning_rates,
    "BiLSTM": True,
    "seeds": seeds,
    "loss": loss,
    "gamma": gamma,
    "device": device,
    "features": features,
    "standardise_method": standardise_method,
    "include_features_in_path": include_features_in_path,
    "include_features_in_input": include_features_in_input,
    "path_indices": client_index,
    "split_ids": client_transcript_id,
    "k_fold": True,
    "patience": patience,
    "validation_metric": validation_metric,
    "verbose": False,
}

# w=5

In [11]:
size = 5

## umap

In [12]:
(
    swnu_network_umap_kfold_5,
    best_swnu_network_umap_kfold_5,
    _,
    __,
) = swnu_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swnu_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swnu_network_umap_focal_2_5_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swnu_network_umap_focal_2_5_kfold_best_model.csv


In [13]:
swnu_network_umap_kfold_5.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_2296920/1104105075.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_umap_kfold_5.groupby(


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (32, 32)       0.1          0.0001         0.641762   
                                                       0.0003         0.643295   
                                                       0.0005         0.641315   
                           (128, 128)     0.1          0.0001         0.641699   
                                                       0.0003         0.636973   
                                                       0.0005         0.634994   
                           (512, 512)     0.1          0.0001         0.633014   
                                                       0.0003         0.624138   
                                                       0.0005         0.625096   
           (12,)           (32, 32)       0.1          0.0001         0.639591   
                                                       0.0003         0.630651   
                                                       0.0005         0.628416   
                           (128, 128)     0.1          0.0001         0.641443   
                                                       0.0003         0.638059   
                                                       0.0005         0.628097   
                           (512, 512)     0.1          0.0001         0.631737   
                                                       0.0003         0.613346   
                                                       0.0005         0.621456   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (32, 32)       0.1          0.0001         0.436265   
                                                       0.0003         0.499392   
                                                       0.0005         0.509760   
                           (128, 128)     0.1          0.0001         0.494475   
                                                       0.0003         0.502848   
                                                       0.0005         0.512454   
                           (512, 512)     0.1          0.0001         0.490158   
                                                       0.0003         0.486104   
                                                       0.0005         0.501809   
           (12,)           (32, 32)       0.1          0.0001         0.449070   
                                                       0.0003         0.485120   
                                                       0.0005         0.494003   
                           (128, 128)     0.1          0.0001         0.501885   
                                                       0.0003         0.498090   
                                                       0.0005         0.511561   
                           (512, 512)     0.1          0.0001         0.482053   
                                                       0.0003         0.495093   
                                                       0.0005         0.508017   

                                                                      precision  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate              
15         (10,)           (32, 32)       0.1          0.0001          0.476034   
                                                       0.0003          0.500995   
                                                       0.0005          0.506743   
                           (128, 128)     0.1          0.0001          0.500957   
                                                       0.0003          0.500741   
                                                       0.0005          0.505986   
                           (512, 512)     0.1          0.0001          0.495206   
                                         

In [14]:
swnu_network_umap_kfold_5.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_2296920/1104105075.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_umap_kfold_5.groupby(


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (32, 32)       0.1          0.0001         0.641762   
                                                       0.0003         0.643295   
                                                       0.0005         0.641315   
                           (128, 128)     0.1          0.0001         0.641699   
                                                       0.0003         0.636973   
                                                       0.0005         0.634994   
                           (512, 512)     0.1          0.0001         0.633014   
                                                       0.0003         0.624138   
                                                       0.0005         0.625096   
           (12,)           (32, 32)       0.1          0.0001         0.639591   
                                                       0.0003         0.630651   
                                                       0.0005         0.628416   
                           (128, 128)     0.1          0.0001         0.641443   
                                                       0.0003         0.638059   
                                                       0.0005         0.628097   
                           (512, 512)     0.1          0.0001         0.631737   
                                                       0.0003         0.613346   
                                                       0.0005         0.621456   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (32, 32)       0.1          0.0001         0.436265   
                                                       0.0003         0.499392   
                                                       0.0005         0.509760   
                           (128, 128)     0.1          0.0001         0.494475   
                                                       0.0003         0.502848   
                                                       0.0005         0.512454   
                           (512, 512)     0.1          0.0001         0.490158   
                                                       0.0003         0.486104   
                                                       0.0005         0.501809   
           (12,)           (32, 32)       0.1          0.0001         0.449070   
                                                       0.0003         0.485120   
                                                       0.0005         0.494003   
                           (128, 128)     0.1          0.0001         0.501885   
                                                       0.0003         0.498090   
                                                       0.0005         0.511561   
                           (512, 512)     0.1          0.0001         0.482053   
                                                       0.0003         0.495093   
                                                       0.0005         0.508017   

                                                                      precision  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate              
15         (10,)           (32, 32)       0.1          0.0001          0.476034   
                                                       0.0003          0.500995   
                                                       0.0005          0.506743   
                           (128, 128)     0.1          0.0001          0.500957   
                                                       0.0003          0.500741   
                                                       0.0005          0.505986   
                           (512, 512)     0.1          0.0001          0.495206   
                                         

In [15]:
best_swnu_network_umap_kfold_5

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.634100,0.512577,"[0.7603625018570792, 0.4419661296984717, 0.335...",0.505683,"[0.7818515123739689, 0.4425144747725393, 0.292...",0.524723,"[0.7400231347599768, 0.4414191419141914, 0.392...",None,0.659762,...,1,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.627969,0.511225,"[0.7537290715372906, 0.4608294930875576, 0.319...",0.502929,"[0.7956298200514139, 0.43103448275862066, 0.28...",0.526114,"[0.7160208212839791, 0.49504950495049505, 0.36...",None,0.650771,...,12,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.622222,0.510882,"[0.745814307458143, 0.4444444444444445, 0.3423...",0.502976,"[0.7872750642673522, 0.4088704088704089, 0.312...",0.524494,"[0.708502024291498, 0.4867986798679868, 0.3781...",None,0.659120,...,123,True,focal,2,True,5,Conv1d,None,concatenation,64


in path (with speaker): 0.5143628505176907

in path (no speaker): 0.522911065406166

in input (no speaker): 0.5125295634373418

both (no speaker): 0.5151677550731781

(w/ time_encoding) in path (with speaker): 0.5186878521492474

(w/ time_encoding) in path (no speaker): 0.5191477428145815

(w/ time_encoding) in input (no speaker): 0.5126337587709865

(w/ time_encoding) both (no speaker):  0.5197361095991764

In [16]:
best_swnu_network_umap_kfold_5["f1"].mean()

0.5115613144357157

In [17]:
best_swnu_network_umap_kfold_5["precision"].mean()

0.503862616648747

In [18]:
best_swnu_network_umap_kfold_5["recall"].mean()

0.5251105694722161

In [19]:
np.stack(best_swnu_network_umap_kfold_5["f1_scores"]).mean(axis=0)

array([0.75330196, 0.44908002, 0.33230196])

In [20]:
np.stack(best_swnu_network_umap_kfold_5["precision_scores"]).mean(axis=0)

array([0.78825213, 0.42747312, 0.2958626 ])

In [21]:
np.stack(best_swnu_network_umap_kfold_5["recall_scores"]).mean(axis=0)

array([0.72151533, 0.47442244, 0.37939394])

# w=11

In [22]:
size = 11

## umap

In [23]:
(
    swnu_network_umap_kfold_11,
    best_swnu_network_umap_kfold_11,
    _,
    __,
) = swnu_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swnu_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swnu_network_umap_focal_2_11_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swnu_network_umap_focal_2_11_kfold_best_model.csv


In [24]:
swnu_network_umap_kfold_11.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_2296920/2463364505.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_umap_kfold_11.groupby(


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (32, 32)       0.1          0.0001         0.630140   
                                                       0.0003         0.624521   
                                                       0.0005         0.626501   
                           (128, 128)     0.1          0.0001         0.643103   
                                                       0.0003         0.640102   
                                                       0.0005         0.638123   
                           (512, 512)     0.1          0.0001         0.620945   
                                                       0.0003         0.622350   
                                                       0.0005         0.624457   
           (12,)           (32, 32)       0.1          0.0001         0.635504   
                                                       0.0003         0.624585   
                                                       0.0005         0.618263   
                           (128, 128)     0.1          0.0001         0.644189   
                                                       0.0003         0.634163   
                                                       0.0005         0.631162   
                           (512, 512)     0.1          0.0001         0.632248   
                                                       0.0003         0.625607   
                                                       0.0005         0.626564   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (32, 32)       0.1          0.0001         0.415930   
                                                       0.0003         0.476840   
                                                       0.0005         0.493381   
                           (128, 128)     0.1          0.0001         0.496153   
                                                       0.0003         0.508833   
                                                       0.0005         0.503703   
                           (512, 512)     0.1          0.0001         0.472357   
                                                       0.0003         0.486478   
                                                       0.0005         0.494355   
           (12,)           (32, 32)       0.1          0.0001         0.437430   
                                                       0.0003         0.459592   
                                                       0.0005         0.476719   
                           (128, 128)     0.1          0.0001         0.496102   
                                                       0.0003         0.505662   
                                                       0.0005         0.501230   
                           (512, 512)     0.1          0.0001         0.484775   
                                                       0.0003         0.485340   
                                                       0.0005         0.491141   

                                                                      precision  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate              
15         (10,)           (32, 32)       0.1          0.0001          0.449824   
                                                       0.0003          0.477912   
                                                       0.0005          0.490460   
                           (128, 128)     0.1          0.0001          0.503176   
                                                       0.0003          0.506110   
                                                       0.0005          0.503089   
                           (512, 512)     0.1          0.0001          0.475959   
                                         

In [25]:
swnu_network_umap_kfold_11.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_2296920/2463364505.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_umap_kfold_11.groupby(


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (32, 32)       0.1          0.0001         0.630140   
                                                       0.0003         0.624521   
                                                       0.0005         0.626501   
                           (128, 128)     0.1          0.0001         0.643103   
                                                       0.0003         0.640102   
                                                       0.0005         0.638123   
                           (512, 512)     0.1          0.0001         0.620945   
                                                       0.0003         0.622350   
                                                       0.0005         0.624457   
           (12,)           (32, 32)       0.1          0.0001         0.635504   
                                                       0.0003         0.624585   
                                                       0.0005         0.618263   
                           (128, 128)     0.1          0.0001         0.644189   
                                                       0.0003         0.634163   
                                                       0.0005         0.631162   
                           (512, 512)     0.1          0.0001         0.632248   
                                                       0.0003         0.625607   
                                                       0.0005         0.626564   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (32, 32)       0.1          0.0001         0.415930   
                                                       0.0003         0.476840   
                                                       0.0005         0.493381   
                           (128, 128)     0.1          0.0001         0.496153   
                                                       0.0003         0.508833   
                                                       0.0005         0.503703   
                           (512, 512)     0.1          0.0001         0.472357   
                                                       0.0003         0.486478   
                                                       0.0005         0.494355   
           (12,)           (32, 32)       0.1          0.0001         0.437430   
                                                       0.0003         0.459592   
                                                       0.0005         0.476719   
                           (128, 128)     0.1          0.0001         0.496102   
                                                       0.0003         0.505662   
                                                       0.0005         0.501230   
                           (512, 512)     0.1          0.0001         0.484775   
                                                       0.0003         0.485340   
                                                       0.0005         0.491141   

                                                                      precision  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate              
15         (10,)           (32, 32)       0.1          0.0001          0.449824   
                                                       0.0003          0.477912   
                                                       0.0005          0.490460   
                           (128, 128)     0.1          0.0001          0.503176   
                                                       0.0003          0.506110   
                                                       0.0005          0.503089   
                           (512, 512)     0.1          0.0001          0.475959   
                                         

In [26]:
best_swnu_network_umap_kfold_11

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.630077,0.501879,"[0.7587131367292226, 0.43706563706563706, 0.30...",0.497777,"[0.7822481572481572, 0.4107402031930334, 0.300...",0.507850,"[0.736552920763447, 0.466996699669967, 0.32]",None,0.659281,...,1,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.647893,0.517412,"[0.7741172994533904, 0.45741324921135645, 0.32...",0.513493,"[0.7913017215342797, 0.4380664652567976, 0.311...",0.522373,"[0.7576633892423366, 0.47854785478547857, 0.33...",None,0.665061,...,12,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.642337,0.507209,"[0.7680928208846991, 0.43148450244698205, 0.32...",0.507059,"[0.7704393366307827, 0.4266129032258065, 0.324...",0.507410,"[0.7657605552342395, 0.43646864686468645, 0.32]",None,0.669236,...,123,True,focal,2,True,5,Conv1d,None,concatenation,64


In [27]:
best_swnu_network_umap_kfold_11["f1"].mean()

0.5088333390935932

In [28]:
best_swnu_network_umap_kfold_11["precision"].mean()

0.5061096028145416

In [29]:
best_swnu_network_umap_kfold_11["recall"].mean()

0.5125443508299162

In [30]:
np.stack(best_swnu_network_umap_kfold_11["f1_scores"]).mean(axis=0)

array([0.76697442, 0.4419878 , 0.3175378 ])

In [31]:
np.stack(best_swnu_network_umap_kfold_11["precision_scores"]).mean(axis=0)

array([0.78132974, 0.42513986, 0.31185921])

In [32]:
np.stack(best_swnu_network_umap_kfold_11["recall_scores"]).mean(axis=0)

array([0.75332562, 0.46067107, 0.32363636])

# w=20

In [33]:
size = 20

## umap

In [34]:
(
    swnu_network_umap_kfold_20,
    best_swnu_network_umap_kfold_20,
    _,
    __,
) = swnu_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swnu_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swnu_network_umap_focal_2_20_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swnu_network_umap_focal_2_20_kfold_best_model.csv


In [35]:
swnu_network_umap_kfold_20.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_2296920/29893126.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_umap_kfold_20.groupby(


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (32, 32)       0.1          0.0001         0.627714   
                                                       0.0003         0.626245   
                                                       0.0005         0.627075   
                           (128, 128)     0.1          0.0001         0.635057   
                                                       0.0003         0.634610   
                                                       0.0005         0.637995   
                           (512, 512)     0.1          0.0001         0.627842   
                                                       0.0003         0.605172   
                                                       0.0005         0.631865   
           (12,)           (32, 32)       0.1          0.0001         0.625479   
                                                       0.0003         0.622669   
                                                       0.0005         0.625926   
                           (128, 128)     0.1          0.0001         0.633780   
                                                       0.0003         0.624010   
                                                       0.0005         0.618263   
                           (512, 512)     0.1          0.0001         0.627714   
                                                       0.0003         0.622925   
                                                       0.0005         0.618327   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (32, 32)       0.1          0.0001         0.423338   
                                                       0.0003         0.471960   
                                                       0.0005         0.486564   
                           (128, 128)     0.1          0.0001         0.472277   
                                                       0.0003         0.489684   
                                                       0.0005         0.506080   
                           (512, 512)     0.1          0.0001         0.475855   
                                                       0.0003         0.481554   
                                                       0.0005         0.489203   
           (12,)           (32, 32)       0.1          0.0001         0.427402   
                                                       0.0003         0.467448   
                                                       0.0005         0.479986   
                           (128, 128)     0.1          0.0001         0.475968   
                                                       0.0003         0.483366   
                                                       0.0005         0.481729   
                           (512, 512)     0.1          0.0001         0.481571   
                                                       0.0003         0.482070   
                                                       0.0005         0.477747   

                                                                      precision  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate              
15         (10,)           (32, 32)       0.1          0.0001          0.456805   
                                                       0.0003          0.481839   
                                                       0.0005          0.488590   
                           (128, 128)     0.1          0.0001          0.489414   
                                                       0.0003          0.495780   
                                                       0.0005          0.509665   
                           (512, 512)     0.1          0.0001          0.479184   
                                         

In [36]:
swnu_network_umap_kfold_20.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_2296920/29893126.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_umap_kfold_20.groupby(


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (32, 32)       0.1          0.0001         0.627714   
                                                       0.0003         0.626245   
                                                       0.0005         0.627075   
                           (128, 128)     0.1          0.0001         0.635057   
                                                       0.0003         0.634610   
                                                       0.0005         0.637995   
                           (512, 512)     0.1          0.0001         0.627842   
                                                       0.0003         0.605172   
                                                       0.0005         0.631865   
           (12,)           (32, 32)       0.1          0.0001         0.625479   
                                                       0.0003         0.622669   
                                                       0.0005         0.625926   
                           (128, 128)     0.1          0.0001         0.633780   
                                                       0.0003         0.624010   
                                                       0.0005         0.618263   
                           (512, 512)     0.1          0.0001         0.627714   
                                                       0.0003         0.622925   
                                                       0.0005         0.618327   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (32, 32)       0.1          0.0001         0.423338   
                                                       0.0003         0.471960   
                                                       0.0005         0.486564   
                           (128, 128)     0.1          0.0001         0.472277   
                                                       0.0003         0.489684   
                                                       0.0005         0.506080   
                           (512, 512)     0.1          0.0001         0.475855   
                                                       0.0003         0.481554   
                                                       0.0005         0.489203   
           (12,)           (32, 32)       0.1          0.0001         0.427402   
                                                       0.0003         0.467448   
                                                       0.0005         0.479986   
                           (128, 128)     0.1          0.0001         0.475968   
                                                       0.0003         0.483366   
                                                       0.0005         0.481729   
                           (512, 512)     0.1          0.0001         0.481571   
                                                       0.0003         0.482070   
                                                       0.0005         0.477747   

                                                                      precision  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate              
15         (10,)           (32, 32)       0.1          0.0001          0.456805   
                                                       0.0003          0.481839   
                                                       0.0005          0.488590   
                           (128, 128)     0.1          0.0001          0.489414   
                                                       0.0003          0.495780   
                                                       0.0005          0.509665   
                           (512, 512)     0.1          0.0001          0.479184   
                                         

In [37]:
best_swnu_network_umap_kfold_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.629694,0.504187,"[0.7584758475847584, 0.4401515151515152, 0.313...",0.499896,"[0.7880299251870324, 0.4068627450980392, 0.304...",0.511356,"[0.7310584152689416, 0.4793729372937294, 0.323...",None,0.668915,...,1,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.609579,0.467723,"[0.7446964155084126, 0.38384615384615384, 0.27...",0.472145,"[0.7536274800118449, 0.3595100864553314, 0.303...",0.466200,"[0.7359745517640255, 0.41171617161716173, 0.25...",None,0.656230,...,12,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.622989,0.483495,"[0.7581374981754488, 0.39060665362035224, 0.30...",0.486423,"[0.7653993516062482, 0.37155621742367834, 0.32...",0.482122,"[0.7510121457489879, 0.41171617161716173, 0.28...",None,0.657836,...,123,True,focal,2,True,5,Conv1d,None,concatenation,64


In [38]:
best_swnu_network_umap_kfold_20["f1"].mean()

0.48513497139300443

In [39]:
best_swnu_network_umap_kfold_20["precision"].mean()

0.48615456435706667

In [40]:
best_swnu_network_umap_kfold_20["recall"].mean()

0.4865591346102029

In [41]:
np.stack(best_swnu_network_umap_kfold_20["f1_scores"]).mean(axis=0)

array([0.75376992, 0.40486811, 0.29676689])

In [42]:
np.stack(best_swnu_network_umap_kfold_20["precision_scores"]).mean(axis=0)

array([0.76901892, 0.37930968, 0.31013509])

In [43]:
np.stack(best_swnu_network_umap_kfold_20["recall_scores"]).mean(axis=0)

array([0.73934837, 0.43426843, 0.28606061])

# w=35

In [44]:
size = 35

## umap

In [45]:
(
    swnu_network_umap_kfold_35,
    best_swnu_network_umap_kfold_35,
    _,
    __,
) = swnu_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swnu_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swnu_network_umap_focal_2_35_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swnu_network_umap_focal_2_35_kfold_best_model.csv


In [46]:
swnu_network_umap_kfold_35.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_2296920/16469293.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_umap_kfold_35.groupby(


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (32, 32)       0.1          0.0001         0.623436   
                                                       0.0003         0.617433   
                                                       0.0005         0.625223   
                           (128, 128)     0.1          0.0001         0.629821   
                                                       0.0003         0.628544   
                                                       0.0005         0.615773   
                           (512, 512)     0.1          0.0001         0.615900   
                                                       0.0003         0.613857   
                                                       0.0005         0.607791   
           (12,)           (32, 32)       0.1          0.0001         0.608748   
                                                       0.0003         0.605492   
                                                       0.0005         0.607280   
                           (128, 128)     0.1          0.0001         0.614496   
                                                       0.0003         0.607791   
                                                       0.0005         0.614049   
                           (512, 512)     0.1          0.0001         0.615134   
                                                       0.0003         0.603768   
                                                       0.0005         0.608812   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (32, 32)       0.1          0.0001         0.414507   
                                                       0.0003         0.456184   
                                                       0.0005         0.461487   
                           (128, 128)     0.1          0.0001         0.474349   
                                                       0.0003         0.471223   
                                                       0.0005         0.480975   
                           (512, 512)     0.1          0.0001         0.465669   
                                                       0.0003         0.480127   
                                                       0.0005         0.477707   
           (12,)           (32, 32)       0.1          0.0001         0.397851   
                                                       0.0003         0.435892   
                                                       0.0005         0.467067   
                           (128, 128)     0.1          0.0001         0.448465   
                                                       0.0003         0.467644   
                                                       0.0005         0.471811   
                           (512, 512)     0.1          0.0001         0.459530   
                                                       0.0003         0.466944   
                                                       0.0005         0.475653   

                                                                      precision  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate              
15         (10,)           (32, 32)       0.1          0.0001          0.449130   
                                                       0.0003          0.462461   
                                                       0.0005          0.469617   
                           (128, 128)     0.1          0.0001          0.481106   
                                                       0.0003          0.480009   
                                                       0.0005          0.478550   
                           (512, 512)     0.1          0.0001          0.467888   
                                         

In [47]:
swnu_network_umap_kfold_35.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_2296920/16469293.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_umap_kfold_35.groupby(


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (32, 32)       0.1          0.0001         0.623436   
                                                       0.0003         0.617433   
                                                       0.0005         0.625223   
                           (128, 128)     0.1          0.0001         0.629821   
                                                       0.0003         0.628544   
                                                       0.0005         0.615773   
                           (512, 512)     0.1          0.0001         0.615900   
                                                       0.0003         0.613857   
                                                       0.0005         0.607791   
           (12,)           (32, 32)       0.1          0.0001         0.608748   
                                                       0.0003         0.605492   
                                                       0.0005         0.607280   
                           (128, 128)     0.1          0.0001         0.614496   
                                                       0.0003         0.607791   
                                                       0.0005         0.614049   
                           (512, 512)     0.1          0.0001         0.615134   
                                                       0.0003         0.603768   
                                                       0.0005         0.608812   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (32, 32)       0.1          0.0001         0.414507   
                                                       0.0003         0.456184   
                                                       0.0005         0.461487   
                           (128, 128)     0.1          0.0001         0.474349   
                                                       0.0003         0.471223   
                                                       0.0005         0.480975   
                           (512, 512)     0.1          0.0001         0.465669   
                                                       0.0003         0.480127   
                                                       0.0005         0.477707   
           (12,)           (32, 32)       0.1          0.0001         0.397851   
                                                       0.0003         0.435892   
                                                       0.0005         0.467067   
                           (128, 128)     0.1          0.0001         0.448465   
                                                       0.0003         0.467644   
                                                       0.0005         0.471811   
                           (512, 512)     0.1          0.0001         0.459530   
                                                       0.0003         0.466944   
                                                       0.0005         0.475653   

                                                                      precision  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate              
15         (10,)           (32, 32)       0.1          0.0001          0.449130   
                                                       0.0003          0.462461   
                                                       0.0005          0.469617   
                           (128, 128)     0.1          0.0001          0.481106   
                                                       0.0003          0.480009   
                                                       0.0005          0.478550   
                           (512, 512)     0.1          0.0001          0.467888   
                                         

In [48]:
best_swnu_network_umap_kfold_35

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.614368,0.481542,"[0.7542197216464318, 0.38433292533659735, 0.30...",0.476271,"[0.772754854368932, 0.3801452784503632, 0.2759...",0.489601,"[0.736552920763447, 0.3886138613861386, 0.3436...",None,0.676461,...,1,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.607471,0.481437,"[0.7399999999999999, 0.4018875344081793, 0.302...",0.476109,"[0.7646514497223936, 0.3839218632607062, 0.279...",0.489199,"[0.7168883747831116, 0.42161716171617164, 0.32...",None,0.654624,...,12,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.621648,0.483291,"[0.7536531365313653, 0.416793893129771, 0.2794...",0.484135,"[0.7696713898100693, 0.3877840909090909, 0.294...",0.484746,"[0.738288027761712, 0.4504950495049505, 0.2654...",None,0.671965,...,123,True,focal,2,True,5,Conv1d,None,concatenation,64


In [49]:
best_swnu_network_umap_kfold_35["f1"].mean()

0.4820898495935779

In [50]:
best_swnu_network_umap_kfold_35["precision"].mean()

0.47883817055794653

In [51]:
best_swnu_network_umap_kfold_35["recall"].mean()

0.48784857934415

In [52]:
np.stack(best_swnu_network_umap_kfold_35["f1_scores"]).mean(axis=0)

array([0.74929095, 0.40100478, 0.29597381])

In [53]:
np.stack(best_swnu_network_umap_kfold_35["precision_scores"]).mean(axis=0)

array([0.7690259 , 0.38395041, 0.2835382 ])

# w=80

In [54]:
size = 80

## umap

In [ ]:
(
    swnu_network_umap_kfold_80,
    best_swnu_network_umap_kfold_80,
    _,
    __,
) = swnu_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swnu_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
swnu_network_umap_kfold_80.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

In [ ]:
swnu_network_umap_kfold_80.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

In [ ]:
best_swnu_network_umap_kfold_80

In [ ]:
best_swnu_network_umap_kfold_80["f1"].mean()

In [ ]:
best_swnu_network_umap_kfold_80["precision"].mean()

In [ ]:
best_swnu_network_umap_kfold_80["recall"].mean()

In [ ]:
np.stack(best_swnu_network_umap_kfold_80["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_umap_kfold_80["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_umap_kfold_80["recall_scores"]).mean(axis=0)

# w=110

In [ ]:
size = 110

## umap

In [ ]:
(
    swnu_network_umap_kfold_110,
    best_swnu_network_umap_kfold_110,
    _,
    __,
) = swnu_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swnu_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

In [ ]:
swnu_network_umap_kfold_110.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

In [ ]:
swnu_network_umap_kfold_110.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

In [ ]:
best_swnu_network_umap_kfold_110

In [ ]:
best_swnu_network_umap_kfold_110["f1"].mean()

In [ ]:
best_swnu_network_umap_kfold_110["precision"].mean()

In [ ]:
best_swnu_network_umap_kfold_110["recall"].mean()

In [ ]:
np.stack(best_swnu_network_umap_kfold_110["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_umap_kfold_110["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_umap_kfold_110["recall_scores"]).mean(axis=0)